# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# import libraries

import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import FeatureUnion, Pipeline


In [4]:
import os
os.path.abspath(os.getcwd())

'/home/workspace'

### Loading Up DATABASE 'disaster_response' Prepared from ETL Stage

In [5]:
# load data from database
#def load_data(data_file)
def load_data():
    engine = create_engine('sqlite:///disaster_response.db')
    conn = engine.connect()
    df = pd.read_sql_table('disaster_response', conn)
    #df.head(2)
    X = df.message.values
    
    y = df.iloc[:, 4:]
    ##y.dropna(axis = 0, how = 'any', inplace=True)
    ##y.fillna(0, inplace=True)
    colnames = y.columns
    #y = y.values
    engine.dispose()
    
    return X, y, colnames

### Starting Verb Extractor

Starting Verb Extractor Function is creating a Few NAN values, that is why avoiding its use for now

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(word_tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0][0], pos_tags[0][1]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return 1

            return 0

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)

        return pd.DataFrame(X_tagged)

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    tokens_wihtout_sw = [w for w in tokens if w not in stopwords.words("english") ]
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    
    for tok in tokens_wihtout_sw:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

** Feature Union Enables Parallel Execution of Code - Removed

In [7]:
def model_pipeline():
    pipeline = Pipeline([
        ('trans_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)), 
            ('tfidf', TfidfTransformer())
        ])),
        ('mclf', MultiOutputClassifier(RandomForestClassifier()))
#        ('clf', RandomForestClassifier())
    ])
    
    return pipeline

model-pipeline_with_sw to be used when using verb extracting function

In [ ]:
def model_pipeline_with_SV():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),
        
        #('mclf', RandomForestClassifier())
        ('mclf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
#def display_results(y_test, y_pred):
#    labels = np.unique(y_pred)
#    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
#    accuracy = (y_pred == y_test).mean()

#    print("Labels:", labels)
#    print("Confusion Matrix:\n", confusion_mat)
#    print("Accuracy:", accuracy)

In [8]:
def train():
    
    X, y, column_names = load_data()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)

    model = model_pipeline()

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
#    display_results(y_test, y_pred)
    return y_test, y_pred, column_names

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_test, y_pred, column_names = train()

In [10]:
target_dataframe = pd.DataFrame(y_pred, columns = column_names)

In [22]:
for i, value in enumerate(target_dataframe):
    print(classification_report(y_test.iloc[:,i], target_dataframe.iloc[:,i], target_names = list(column_names)))

                        precision    recall  f1-score   support

               related       0.31      0.11      0.16      1555
               request       0.76      0.93      0.84      4955
                 offer       0.25      0.02      0.04        42

           avg / total       0.65      0.73      0.67      6552

                        precision    recall  f1-score   support

               related       0.84      0.97      0.90      5454
               request       0.32      0.06      0.10      1098

           avg / total       0.75      0.82      0.77      6552

                        precision    recall  f1-score   support

               related       1.00      1.00      1.00      6527
               request       0.00      0.00      0.00        25

           avg / total       0.99      1.00      0.99      6552

                        precision    recall  f1-score   support

               related       0.60      0.80      0.68      3878
               request       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 3, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 36
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
target_dataframe.shape[1]

36

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.